In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import seaborn as sns

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
import os

In [18]:
Folder = "pw3_data"
Files = ["EEG_mouse_data_1.csv", "EEG_mouse_data_2.csv"]

# Lecture des données
datas = []
for file in Files:
    datas.append(pd.read_csv(Folder + "/" + file))

# Concaténation des données
mouse1_eeg = pd.concat(datas)

# Sélection des features et normalisation des données.

J'ai choisi ici de sélectionner des features de manière à ce qu'elles soient les plus pertinentes possibles pour la prédiction de la variable cible. Pour cela, j'ai utilisé la méthode de sélection de features de type "wrapper" appelée "Recursive Feature Elimination" (RFE) qui consiste à éliminer les features une à une en fonction de leur importance pour la prédiction de la variable cible. J'ai utilisé pour cela un modèle de régression logistique.

In [19]:
# Exemple de sélection de 25 caractéristiques réparties uniformément
selected_features = np.linspace(1, 101, 25, dtype=int)
selected_features = [f'amplitude_around_{i}_Hertz' for i in selected_features]

# Extraction des données correspondant aux caractéristiques sélectionnées
data_selected = mouse1_eeg[selected_features]

# Normalisation des données avec Z-score
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data_selected)

# Conversion en DataFrame pour une utilisation ultérieure
data_normalized_df = pd.DataFrame(data_normalized, columns=selected_features)

# Affichage des 5 premières lignes du DataFrame
print(data_normalized_df.head())

   amplitude_around_1_Hertz  amplitude_around_5_Hertz  \
0                 -0.209955                  0.818127   
1                 -0.183434                  0.740384   
2                 -0.143389                  0.036759   
3                 -0.443261                 -0.510886   
4                 -0.187587                  0.570556   

   amplitude_around_9_Hertz  amplitude_around_13_Hertz  \
0                  0.709286                  -0.133015   
1                  0.099354                  -0.289620   
2                 -0.457418                  -0.426393   
3                 -0.465943                   0.431755   
4                  0.829106                   0.528520   

   amplitude_around_17_Hertz  amplitude_around_21_Hertz  \
0                  -0.356923                   0.046292   
1                   0.893280                   1.456953   
2                   1.615055                  -0.280723   
3                  -0.464970                  -0.643038   
4            

In [20]:
def create_model(input_dim):
    mlp = keras.Sequential([
        layers.Dense(2, activation="tanh", input_shape=(input_dim,)),
        layers.Dense(1, activation="sigmoid"),
    ])

    mlp.compile(
        optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.99),
        loss="binary_crossentropy",
        metrics=['accuracy']
    )
    return mlp

In [46]:
import glob
if( True ):
    # Load the datasets
    data1 = pd.read_csv('EEG_mouse_data_1.csv')
    data2 = pd.read_csv('EEG_mouse_data_2.csv')

    # Combine and preprocess the data
    data_combined = pd.concat([data1, data2])
    data_combined['state'] = data_combined['state'].replace({'n': 0, 'r': 0, 'w': 1})
    X = data_combined.drop('state', axis=1)
    y = data_combined['state']

    # Normalize the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Feature selection with RFE
    model = LogisticRegression(max_iter=10000)
    rfe = RFE(estimator=model, n_features_to_select=25)
    X_rfe = rfe.fit_transform(X_scaled, y)

    y_keras = y.to_numpy()

    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    history_list = []
    trained_mlps = []

    bestmodel = None
    bestAccuracy = 0

    for train_index, test_index in kf.split(X_rfe):

        print(len(train_index))

        mlp = create_model(X_rfe.shape[1])

        # Les données doivent être divisées selon les indices définis par KFold
        X_train_kf, X_test_kf = X_rfe[train_index], X_rfe[test_index]
        y_train_kf, y_test_kf = y_keras[train_index], y_keras[test_index]

        history = mlp.fit(
            X_train_kf, y_train_kf,
            validation_data=(X_test_kf, y_test_kf),
            epochs=100, verbose=1
        )

        print(f"Accuracy: {history.history['val_accuracy'][-1]:.2f}")

        history_list.append(history)
        trained_mlps.append(mlp)

        if history.history['val_accuracy'][-1] > bestAccuracy:
            bestAccuracy = history.history['val_accuracy'][-1]
            bestmodel = mlp
else :

    # Get the highest float value from the filenames
    highest_float = float('-inf')
    files = os.listdir('.')

    model_file = [fichier for fichier in os.listdir('.') if fichier.startswith('m_')]
    model_file_float = [fichier[2:] for fichier in model_file]
    for file in model_file_float:
        float_value = float(file)
        if float_value > highest_float:
            highest_float = float_value

    # Load the model from the highest float filename
    model_filename = 'm_' + str(highest_float)
    model = keras.models.load_model(model_filename)


/tmp/ipykernel_29190/2668360709.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_combined['state'] = data_combined['state'].replace({'n': 0, 'r': 0, 'w': 1})


27242
Epoch 1/100
852/852 [==============================] - 1s 1ms/step - loss: 0.2877 - accuracy: 0.8915 - val_loss: 0.2548 - val_accuracy: 0.9044
Epoch 2/100
852/852 [==============================] - 1s 1ms/step - loss: 0.2513 - accuracy: 0.9011 - val_loss: 0.2564 - val_accuracy: 0.9027
Epoch 3/100
852/852 [==============================] - 1s 1ms/step - loss: 0.2482 - accuracy: 0.8997 - val_loss: 0.2553 - val_accuracy: 0.8886
Epoch 4/100
852/852 [==============================] - 1s 1ms/step - loss: 0.2475 - accuracy: 0.8967 - val_loss: 0.2489 - val_accuracy: 0.8929
Epoch 5/100
852/852 [==============================] - 1s 1ms/step - loss: 0.2468 - accuracy: 0.8974 - val_loss: 0.2501 - val_accuracy: 0.8990
Epoch 6/100
852/852 [==============================] - 1s 1ms/step - loss: 0.2450 - accuracy: 0.8960 - val_loss: 0.2576 - val_accuracy: 0.8906
Epoch 7/100
852/852 [==============================] - 1s 1ms/step - loss: 0.2450 - accuracy: 0.8988 - val_loss: 0.2499 - val_accuracy: 

In [48]:
bestmodel.save(f'm_{bestAccuracy}')


INFO:tensorflow:Assets written to: m_0.9019895792007446/assets


INFO:tensorflow:Assets written to: m_0.9019895792007446/assets
